## Making the EBcache
+ should work similar to EBscore with these differences:
    * no tumor bam is needed

In [ ]:
# get the code
import sys
import os
sys.path.append('../code')

# set the paths
home = '/Users/martinscience'
# home = '/Users/mahtin'


somvar_path = os.path.join(home, "Dropbox/Icke/Work/somVar")
testdata = os.path.join(somvar_path, "testdata")
ebdata = os.path.join(somvar_path, "tooldata/EBdata")
pon_path = os.path.join(testdata, "PON")

static = os.path.join(home, "Dropbox/Icke/Work/static")

## make EBcache matrix
+ to be used by bam2matrix in cache-mode

from ebcache import PON2matrix

EBconfig = {
    "temp_dir": os.path.join(ebdata, "temp"),
    "pon_path": pon_path,
    "zero_path": os.path.join(pon_path, "zero"),
    "bed_file": os.path.join(static, "bed_files/SureSelect/hg38/SS_HAEv7_hg38_Padded.bed"),
    "mawk_path": "../shell",
    "genome_split": f"{static}/genome/gatk/hg38/split",
    "MAPQ": 20,
    "Q": 25,
    "fit_pen": 0.5,
    "use_cache": True
}
pon_list = "PON_chr19.txt"
chrom = "chr19"

PON2matrix(pon_list, chrom, EBconfig)

In [ ]:
chr7_df = pd.read_csv(os.path.join(pon_path, "matrix/chr7.pon.gz"), compression="gzip", sep="\t")
chr7_df

## computation has  to be done per PON string
+ allows use of same function for both PONcache and matrix2AB
+ allows optimal caching of the results

In [ ]:
from ebcache import PONmatrix2AB_multi
from zerocache import collapse_zeros

_ = collapse_zeros(os.path.join(pon_path, "zero"))

PONAB_config = {
    "fit_pen": 0.5,
    "threads": 8,
    "chunk_size":5000,
    "pon_path": pon_path,
    "zero_condense_factor": 13 # how much complexity remains after flattening the tumor-zero lines
}
test_df = chr7_df.iloc[:10000, :]
chr7_AB_df = PONmatrix2AB_multi(test_df, config=PONAB_config)
chr7_AB_df

In [ ]:
try:
    pd.read_csv("/Users/mahtin/Dropbox/Icke/Work/somVar/testdata/PON/zero/zer.csv", sep="\t")
except:
    print("Does not work")

In [ ]:
chr7_AB_df.to_csv(os.path.join(pon_path, "ABcache/chr7.AB"), sep="\t", index=False)

In [ ]:
from zerocache import collapse_zeros
zdf = collapse_zeros(os.path.join(pon_path, "zero"), zero_condense_factor=13)
zdf